In [1]:
#ALWAYS RUN FIRST!

#Code that tries to set the largest csv field size.

def max_csv_field_size():
    maxInt = sys.maxsize
    decrement = True
    while decrement:
        decrement = False
        try:
            csv.field_size_limit(maxInt)
        except OverflowError:
            maxInt = int(maxInt/10)
            decrement = True

#Import Libraries
            
import re
from pylab import *
import csv
import sys
max_csv_field_size()
import psycopg2
import spacy
spacy.load('en')
from spacy.lang.en import English
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import pickle
import math
import warnings
import gensim
from gensim import corpora
from gensim.models.keyedvectors import KeyedVectors
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import os
        
#Set the names of the files in which you want to save data.

profile_data_name = 'profile_search_profile_data'
question_data_name = 'profile_search_question_data'
meeting_data_name = 'profile_search_meeting_data'
event_data_name = 'profile_search_event_data'
event_attended_data_name = 'profile_search_event_attended_data'

#Set some global variables, the language of the parser and the word2vec library.

parser = English()
google_word2vec_model = KeyedVectors.load_word2vec_format(os.getcwd() + '\data\GoogleNews-vectors-negative300.bin', binary=True)

C:\Users\Alexander\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Alexander\\Desktop\\data\\GoogleNews-vectors-negative300.bin'

In [3]:
#Extract the data from the CRM. Don't run if you have an up to date copy of the CRM.

try:
    
    #Opens connection to the CRM asks for peoples name and description.
    #Output rows for row in rows row[0] - first name, row[1] - second name, row[2] - description.
    
    print("Trying to access CRM CSaP database ...")
    
    conn = psycopg2.connect(SERVER_INFO)
    
    cur = conn.cursor()
    
    cur.execute("""SELECT
    person.id,
    person.first_name,
    person.last_name,
    person.description
    FROM people_person as person
    ;
    """)
    rows = cur.fetchall()
    
    #Saves data to the file called above.
    
    with open(os.getcwd() + '\data\\' + profile_data_name + '.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            wr.writerow([str(row[0]), row[1].encode('utf-8'), row[2].encode('utf-8'), row[3].encode('utf-8')])
                         
    cur.execute("""SELECT
    academic.id,
    people_policyfellowshipschedule.policy_fellowship_id
    FROM people_person as academic
    JOIN people_policyfellowshipschedule_people ON people_policyfellowshipschedule_people.person_id = academic.id
    JOIN people_policyfellowshipschedule ON people_policyfellowshipschedule_people.policyfellowshipschedule_id = people_policyfellowshipschedule.id
    ;
    """)
    rows = cur.fetchall()
    
    with open(os.getcwd() + '\data\\' + meeting_data_name + '.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            wr.writerow([str(row[0]), str(row[1])])
                         
    cur.execute("""SELECT
    people_policyfellowship.id,
    fellow.first_name,
    fellow.last_name,
    people_policyfellowship.description,
    question.question
    FROM people_policyfellowship
    LEFT JOIN people_policyfellowshipquestion as question ON people_policyfellowship.id = question.policy_fellowship_id
    JOIN people_person as fellow ON fellow.id = people_policyfellowship.policy_fellow_id
    ;
    """)
    rows = cur.fetchall()
    
    #Saves data to the file called above.
    
    with open(os.getcwd() + '\data\\' + question_data_name + '.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            if row[4] is None:
                wr.writerow([str(row[0]), row[1].encode('utf-8'), row[2].encode('utf-8'), row[3].encode('utf-8'), ''.encode('utf-8')])
            else:
                wr.writerow([str(row[0]), row[1].encode('utf-8'), row[2].encode('utf-8'), row[3].encode('utf-8'), row[4].encode('utf-8')])
    
    cur.execute("""SELECT
    academic.id,
    events_personeventrole.event_id
    FROM people_person as academic
    JOIN events_personeventrole ON events_personeventrole.person_id = academic.id
    ;
    """)
    rows = cur.fetchall()
    
    #Saves data to the file called above.
    
    with open(os.getcwd() + '\data\\' + event_attended_data_name + '.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            wr.writerow([str(row[0]), str(row[1])])
    
    cur.execute("""SELECT
    event.id,
    event.name,
    event.description
    FROM events_event as event
    ;
    """)
    rows = cur.fetchall()
    
    #Saves data to the file called above.
    
    with open(os.getcwd() + '\data\\' + event_data_name + '.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            wr.writerow([str(row[0]), row[1].encode('utf-8'), row[2].encode('utf-8')])
    
    print("... data downloaded and saved to disk.")    
    
except:
    #If server isn't online this collects data from the save file.
    
    print("... can't access server, is the tunnel set up? Can continue on previously saved data.")

Trying to access CRM CSaP database ...
... data downloaded and saved to disk.


In [4]:
#Functions for cleaning the text and getting it ready to be procced.

#Gets rid of HTML tags and end of line markers.
#Input: String of text from CRM or internet.
#Output: Cleaned up string of text without HTML tags or end of line markers.

def clean_text(text):
    
    #Removes HTML tags.
    
    clean = re.compile('<.*?>')
    temp_text = re.sub(clean, '', text)
    
    #Removes rouge utf-8 code.
    
    clean = re.compile('\\\\x\w\w')
    temp_text = re.sub(clean, '', temp_text)
    
    clean = re.compile('\\\\x\w')
    temp_text = re.sub(clean, '', temp_text)
    
    #Removes end of line indicators and other junk.
    
    tags = ['\\r','\\n','/','\\t','\\']
    
    for tag in tags:
        temp_text = temp_text.replace(tag,'')
    
    return temp_text

#Tokenizes text, seperates it into a string of words and grammar.
#Input: A string of text.
#Output: A list of words and grammar in order all in lower case.

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

#Lemmatiser, this finds the root word (i.e. depluralises).
#Input: a token, i.e. a single word or grammar.
#Output: a lemma which is the base of the word and association 

def get_lemma(word):
    return WordNetLemmatizer().lemmatize(word)

#Checks if two words are synonyms or antonyms.
#Input: Takes two strings word_1 and word_2, they should be prepared words.
#Output: Returns 1 if they are synonyms, -1 if they are antonyms or 0 otherwise.

def is_synonym_antonym(word_1,word_2):
    word_1_synonyms = set([word for synset in wn.synsets(word_1) for word in synset.lemma_names()])
    word_2_synonyms = set([word for synset in wn.synsets(word_2) for word in synset.lemma_names()])
    word_1_antonyms = set([ant.name() for synset in wn.synsets(word_1) for syn in synset.lemmas() for ant in syn.antonyms()])
    word_2_antonyms = set([ant.name() for synset in wn.synsets(word_2) for syn in synset.lemmas() for ant in syn.antonyms()])

    if word_1_synonyms & word_2_synonyms or word_1_antonyms & word_2_antonyms:
        return 1

    if word_1_synonyms & word_2_antonyms or word_2_synonyms & word_1_antonyms:
        return -1
    
    return 0
    
#Prepares text for the analysis, tokenizes texts, gets rid of words length less than 4 and filters out non-useful words then
#Lemmatisers the text.
#Input: A string of text you want to analysis.
#Output: A list of Lemmas of the meaningful words.

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    
    tokens = [tok[0] for tok in nltk.pos_tag(tokens) if tok[1][0] == 'N']
    
    en_stop = set(nltk.corpus.stopwords.words('english'))
    
    tokens = [get_lemma(token) for token in tokens if (len(token) > 4) and (token not in en_stop)]
    return tokens

In [5]:
#This uploads the data into the program from the file, cleaning the data and preparing for the LDA whilst it does it.

print("Trying to load up data ...")

people = {}
policy_fellowships = {}
events = {}

try:
    print('... uploading people, ...')
    
    with open(os.getcwd() + '\data\\' + profile_data_name+'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            profile = clean_text(row[3][2:-1])
            if len(profile) > 200:
                people[int(row[0])] = [clean_text(row[1][2:-1]), clean_text(row[2][2:-1]), prepare_text_for_lda(profile),[],[]]
            else:
                people[int(row[0])] = [clean_text(row[1][2:-1]), clean_text(row[2][2:-1]),'',[],[]]
    
    with open(os.getcwd() + '\data\\' + meeting_data_name +'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            people[int(row[0])][3].append(int(row[1]))
            
    
    with open(os.getcwd() + '\data\\' + event_attended_data_name +'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            people[int(row[0])][4].append(int(row[1]))
    
    print('... uploading questions, ...')
    
    with open(os.getcwd() + '\data\\' + question_data_name +'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            if int(row[0]) in policy_fellowships.keys():
                if row[4] != '':
                    policy_fellowships[int(row[0])][2].append([clean_text(row[4][2:-1]), prepare_text_for_lda(clean_text(row[4][2:-1]))])
            else:
                if row[4] != '':
                    policy_fellowships[int(row[0])] = [clean_text(row[1][2:-1]), clean_text(row[2][2:-1]), [[clean_text(row[3][2:-1]), prepare_text_for_lda(clean_text(row[3][2:-1]))],[clean_text(row[4][2:-1]), prepare_text_for_lda(clean_text(row[4][2:-1]))]]]
                else:
                    policy_fellowships[int(row[0])] = [clean_text(row[1][2:-1]), clean_text(row[2][2:-1]), [[clean_text(row[3][2:-1]), prepare_text_for_lda(clean_text(row[3][2:-1]))]]]
    
    print('... and uploading events, ...')
    
    with open(os.getcwd() + '\data\\' + event_data_name +'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            events[int(row[0])] = [clean_text(row[1][2:-1]), prepare_text_for_lda(clean_text(row[2][2:-1]))]
    
    print("... data successfully uploaded.")
    
except:
    
    print(".. no back up data, please connect to server.")

Trying to load up data ...
... uploading people ...
... uploading questions ...
... uploading events ...
... data successfully uploaded.


In [6]:
#Theme extractor, submit a list of texts and it will extract topic_num themes each summerised by word_num worth of words each
#with individual ratings on how important the word is.
#Input: List of prepared text.
#Output: a list 

def get_themes(text_data, topic_num, word_num):
    dictionary = corpora.Dictionary(text_data)
    
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = topic_num, id2word=dictionary, passes=15)
    ldamodel.save('model5.gensim')
    return ldamodel.print_topics(num_words=word_num)
        

#for theme in get_themes([person[3] for person in people_prepared], 5,4):
#    print(theme)

(0, '0.062*"university" + 0.050*"research" + 0.024*"cambridge" + 0.021*"professor"')
(1, '0.067*"health" + 0.019*"policy" + 0.014*"research" + 0.009*"development"')
(2, '0.027*"director" + 0.026*"business" + 0.017*"management" + 0.017*"cambridge"')
(3, '0.024*"research" + 0.018*"university" + 0.013*"cambridge" + 0.012*"material"')
(4, '0.040*"policy" + 0.030*"research" + 0.024*"science" + 0.019*"university"')


In [12]:
#Checks which words where found to be similar without repeats.
#Input: Two texts as list of strings.
#Output: List of similar words in pairs without repeats, the first appears in text_1, the second in text_2.

def get_similar_words(text_1, text_2):
    similar_words = []
    for word_1 in text_1:
        for word_2 in text_2:
            if is_synonym_antonym(word_1,word_2) != 0:
                if [word_1, word_2] not in similar_words and [word_2, word_1] not in similar_words:
                    similar_words.append([word_1,word_2])
    return(similar_words)

#Set words to be the words you want to compare against, this prepares this text into a nice list. (TO ADD THEME EXTRACTION)

words = 'emerging markets india trade'

prepared_words = prepare_text_for_lda(words)
    
#This searches the list of people for who are relevant to the words you have asked about and orderes them by what proportion of
#the words in there profile fit these words.

relevant_people = []
policy_fellowships_val = {}
events_val = {}

for key in policy_fellowships.keys():
    policy_fellowships_val[key] = 0
    for question in policy_fellowships[key][2]:
        for question_word in question[1]:
            for word in prepared_words:
                policy_fellowships_val[key] += abs(is_synonym_antonym(word,question_word))
                
for key in events.keys():
    events_val[key] = 0
    for event_word in events[key][1]:
        for word in prepared_words:
            events_val[key] += abs(is_synonym_antonym(word,event_word))

for key in people.keys():
    person = people[key]
    profile_count = 0
    policy_fellowship_count = 0
    event_count = 0
    if len(person[2]) > 0:
        for word in prepared_words:
            for profile_word in person[2]:
                profile_count += abs(is_synonym_antonym(word,profile_word))
        profile_count /= len(person[2])
    if len(person[3]) > 0:
        for policy_fellowship in person[3]:
            policy_fellowship_count += policy_fellowships_val[policy_fellowship]
        policy_fellowship_count /= len(person[3])
    if len(person[4]) > 0:
        for event in person[4]:
            event_count += events_val[event]
        event_count /= len(person[4])
    
    if profile_count > 0 or policy_fellowship_count > 0 or event_count > 0:
        relevant_people.append([key, profile_count, policy_fellowship_count, event_count])
        
profile_max = max([person[1] for person in relevant_people])
policy_fellowship_max = max([person[2] for person in relevant_people])
event_max = max([person[3] for person in relevant_people])

relevant_people = sorted(relevant_people,key=lambda person: person[1]/profile_max + person[2]/policy_fellowship_max + person[3]/event_max, reverse = True)

#This prints the top 10 fitting people.

print('We suggest the following people from the CSaP database:')

for i in range(10):
    person = people[relevant_people[i][0]]
    print((str(i+1) +'\\ '), person[0], person[1], 'as ... ')
    if relevant_people[i][1] > 0:
        print('  ... their profile mensions ...' )
        for word_pair in get_similar_words(prepared_words, person[2]):
            if word_pair[0] == word_pair[1]:
                print("      ... '", word_pair[0], "' which also apears in the text.")
            else:
                print("      ... '" + word_pair[1] + "' which is similar to '" + word_pair[0] + "' that appears in the text.")  
    if relevant_people[i][2] > 0:
        print('  ... they met ...')
        for policy_fellowship_key in person[3]:
            if policy_fellowships_val[policy_fellowship_key] != 0:
                fellowship = policy_fellowships[policy_fellowship_key]
                print('    ...', fellowship[0], fellowship[1], 'who talks about ...')
                similar_words = []
                for fellowship_text in fellowship[2]:
                    for word_pair in get_similar_words(prepared_words,fellowship_text[1]):
                        if word_pair not in similar_words and [word_pair[1], word_pair[0]] not in similar_words:
                            similar_words.append(word_pair)
                for word_pair in similar_words:
                    if word_pair[0] == word_pair[1]:
                        print("      ... '", word_pair[0], "' which also apears in the text.")
                    else:
                        print("      ... '" + word_pair[1] + "' which is similar to '" + word_pair[0] + "' that appears in the text.")  
    if relevant_people[i][3] > 0:
        print('  ... they went to ...')
        for event_key in person[4]:
            if events_val[event_key] != 0:
                event = events[event_key]
                print('    ...', event[0], 'which had in the breif ...')
                for word_pair in get_similar_words(prepared_words,event[1]):
                    if word_pair[0] == word_pair[1]:
                        print("      ... '", word_pair[0], "' which also apears in the text.")
                    else:
                        print("      ... '" + word_pair[1] + "' which is similar to '" + word_pair[0] + "' that appears in the text.")       


We suggest the following people from the CSaP database:
1\  Eden Yin as ... 
  ... their profile mensions ...
      ... 'marketing' which is similar to 'market' that appears in the text.
      ... 'business' which is similar to 'trade' that appears in the text.
      ... 'product' which is similar to 'trade' that appears in the text.
  ... they met ...
    ... Chris Floyd who talks about ...
      ... 'business' which is similar to 'trade' that appears in the text.
      ... 'change' which is similar to 'trade' that appears in the text.
    ... Rupert Cryer who talks about ...
      ... 'marketing' which is similar to 'market' that appears in the text.
      ... 'product' which is similar to 'trade' that appears in the text.
      ... 'change' which is similar to 'trade' that appears in the text.
      ... 'business' which is similar to 'trade' that appears in the text.
    ... Louis Barson who talks about ...
      ... ' trade ' which also apears in the text.
      ... 'business' whic

In [23]:
#Given a dictionary of word weights returns the value of the text in sum form, no normalisation. 
#to_one = False - sums the values of each word or phrase.
#to_one = True - checks if words from the dictionary appears if so adds the weight.
#Input: The text to be evaluated as a list of words in order, word_dictionary with words connected to values, return_words a
#boolean to signify whether to return the words in the output or not, and to_one to signify which counting method to use.

def evaluate_text(text, word_dictionary, return_words = False, to_one = False):
    value = 0
    word_list = []
        
    for i,word in enumerate(text):
        try:
            value += word_dictionary[word]
            word_list.append(word)
        except:
            pass
        
        try:
            value += word_dictionary[word + '_' + text[i+1]]
            word_list.append(word + '_' + text[i+1])
        except:
            pass
        
        try:
            value += word_dictionary[word + '_' + text[i+1] + '_' + text[i+2]]
            word_list.append(word + '_' + text[i+1] + '_' + text[i+2])
        except:
            pass
    
    if to_one:
        for word in set(word_list):
            value -= word_dictionary[word]*(word_list.count(word)-1)
    
    if return_words:
        return value, word_list
    else:
        return value

#Defines a dictionary where words are linked to the minimal cosin distance from a word in words, going up to the number_of_words
#closest words in the Vector space defined by the model.
#Input: model a Words2Vec model, words a list of words/phrases and number_of_words the number of close words you want to consider.
#Output: A dictionary linking words with a value.

def generate_word_dictionary(model, words, number_of_words = 5000):
    word_dictionary = {}
    for word in words:
        word_dictionary[word] = 1
        try:
            for pair in word2vec_model.most_similar(positive=[word],topn=number_of_words):
                if pair[0].lower() in word_dictionary.keys():
                    word_dictionary[pair[0].lower()] = max([pair[1], word_dictionary[pair[0].lower()]])
                else:
                    word_dictionary[pair[0].lower()] = pair[1]
        except:
            print(word, 'not found in dictionary, no similar words discovered.')
    
    return word_dictionary

#Set words to be the words you want to compare against, this prepares this text into a nice list. (TO ADD THEME EXTRACTION)

words = ['emerging_markets']
    
#This searches the list of people for who are relevant to the words you have asked about and orderes them by what proportion of
#the words in there profile fit these words.

word_dictionary = generate_word_dictionary(google_word2vec_model,words)
relevant_people = []
policy_fellowships_val = {}
events_val = {}

for key in policy_fellowships.keys():
    policy_fellowships_val[key] = 0
    for question in policy_fellowships[key][2]:
        policy_fellowships_val[key] += evaluate_text(question[1],word_dictionary)
                
for key in events.keys():
    events_val[key] = evaluate_text(events[key][1],word_dictionary)

for key in people.keys():
    person = people[key]
    profile_count = 0
    policy_fellowship_count = 0
    event_count = 0
    if len(person[2]) > 0:
        profile_count = evaluate_text(person[2],word_dictionary)
    if len(person[3]) > 0:
        for policy_fellowship in person[3]:
            policy_fellowship_count += policy_fellowships_val[policy_fellowship]
        policy_fellowship_count /= len(person[3])
    if len(person[4]) > 0:
        for event in person[4]:
            event_count += events_val[event]
        event_count /= len(person[4])
    
    if profile_count > 0 or policy_fellowship_count > 0 or event_count > 0:
        relevant_people.append([key, profile_count, policy_fellowship_count, event_count])
        
profile_max = max([person[1] for person in relevant_people])
policy_fellowship_max = max([person[2] for person in relevant_people])
event_max = max([person[3] for person in relevant_people])

print('Profile max:',profile_max)
print('PF max:', policy_fellowship_max)
print('Event max:', event_max)

relevant_people = sorted(relevant_people,key=lambda person: person[1]/profile_max + person[2]/policy_fellowship_max + person[3]/event_max, reverse = True)

#This prints the top 10 fitting people.

print('We suggest the following people from the CSaP database:')

for i in range(10):
    person = people[relevant_people[i][0]]
    print((str(i+1) +'\\ '), person[0], person[1])

Profile max: 8.939088106155396
PF max: 12.39812058210373
Event max: 6.110476970672607
We suggest the following people from the CSaP database:
1\  Nikku Madhusudhan 0 3.9846407175064087 6.110476970672607 1.321390705237865
2\  Vasant Kumar 0 3.685674399137497 6.110476970672607 1.297276863435063
3\  Karishma Jain 0 3.3093951046466827 6.110476970672607 1.2669271590585822
4\  Lalita Ramakrishnan 0.40359991788864136 1.7735903561115265 6.110476970672607 1.18820317237784
5\  Anuj Dawar 0 1.3992066085338593 6.110476970672607 1.112856347804325
6\  Neil Prem 0 1.3332746028900146 6.110476970672607 1.107538444561876
7\  Monica Racovita 0.8295448422431946 0.0 6.110476970672607 1.0927997165249972
8\  Mireille El-Hajj 8.939088106155396 0 0.0 1.0
9\  Kate Kirk 0 0 6.110476970672607 1.0
10\  Oliverkhiowboong Heng 0 0 6.110476970672607 1.0


'\n    if relevant_people[i][1] > 0:\n        print(\'  ... their profile mensions ...\' )\n        for word_pair in get_similar_words(prepared_words, person[2]):\n            if word_pair[0] == word_pair[1]:\n                print("      ... \'", word_pair[0], "\' which also apears in the text.")\n            else:\n                print("      ... \'" + word_pair[1] + "\' which is similar to \'" + word_pair[0] + "\' that appears in the text.")  \n    if relevant_people[i][2] > 0:\n        print(\'  ... they met ...\')\n        for policy_fellowship_key in person[3]:\n            if policy_fellowships_val[policy_fellowship_key] != 0:\n                fellowship = policy_fellowships[policy_fellowship_key]\n                print(\'    ...\', fellowship[0], fellowship[1], \'who talks about ...\')\n                similar_words = []\n                for fellowship_text in fellowship[2]:\n                    for word_pair in get_similar_words(prepared_words,fellowship_text[1]):\n          